# Steps
1. Set Translation Language
2. upload one json to translate
3. run all cells
4. download your translation


# TODO:
make simple version?

edge cases:
- unicode characters and UTF8 conversion
- empty values
- broken string handling: single character values,



##Fancier version will:
4. connect open-ai
5. connect gguf
6. remove any \u00e8 type characters
7. check for empty leaf node (as fail json test)

maybe:
- not do whole sections but iterate
item by items crawl...



Version:
1. python script
2. endpoint
3. using api-model service (online)
4. using local .GGUF model (offline)
	- make:
		A. raw llama.cpp wrapper
		B. call_api for python-lamma-cpp
5. swap out, swap back




...

for writeup:
discuss balance of system-1 and system -2
remove any \u00e8 type characters




In [37]:
"""
.env: get your environment variables:
  Using the Google Secretes (like.env) system
  built into colab on the left menu: the 'key' icon.
"""
from google.colab import userdata
mistral_api_key = userdata.get('mistral_api_key')
# mistral_api_key = 'xxx'
openai_api_key = userdata.get('open_ai_key')

# make a list of json files

In [38]:
import os


# Helper Function
def list_json_files_in_cwd():
    """
    Returns a list of .json files in the current working directory.
    """
    # List all files in the current working directory
    files_in_cwd = os.listdir('.')

    # Filter the list to include only .json files
    json_files = [file for file in files_in_cwd if file.endswith('.json')]

    return json_files


# Process Json
- make a model/skeleton template

In [39]:
import json


# Helper Function
def load_json_file(file_path):
    """
    Loads and reads a JSON file, returning its content as a Python dictionary.

    :param file_path: Path to the .json file to be read.
    :return: A dictionary containing the data from the JSON file.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data


# Helper Function
def list_create_deep_empty_structure(data):
    """
    Recursively creates a deep copy of the provided data structure,
    replacing all terminal values with empty lists, and maintaining the same structure.

    :param data: The original data structure, potentially containing nested dictionaries.
    :return: A new data structure with the same keys but with empty lists as terminal values.
    """
    if isinstance(data, dict):  # Checks if the current data item is a dictionary
        return {key: list_create_deep_empty_structure(value) for key, value in data.items()}
    else:
        return []  # Replaces non-dictionary values with an empty list


# Helper Function
def string_create_deep_empty_structure(data):
    """
    Recursively creates a deep copy of the provided data structure,
    replacing all terminal values with empty lists, and maintaining the same structure.

    :param data: The original data structure, potentially containing nested dictionaries.
    :return: A new data structure with the same keys but with empty lists as terminal values.
    """
    if isinstance(data, dict):  # Checks if the current data item is a dictionary
        return {key: string_create_deep_empty_structure(value) for key, value in data.items()}
    else:
        return ""  # Replaces non-dictionary values with an empty list




# Helper Function
def create_empty_json_file(original_data, new_file_path):
    """
    Creates a new JSON file replicating the structure of the original data,
    but with empty lists for all terminal values.

    :param original_data: Dictionary containing the original data.
    :param new_file_path: Path for the new .json file to be created.
    """
    # Create a new structure with the same keys but with empty lists as terminal values
    empty_data = list_create_deep_empty_structure(original_data)

    # Write this new structure to a new JSON file
    with open(new_file_path, 'w') as file:
        json.dump(empty_data, file, indent=4)

    return empty_data


# Helper Function
def create_empty_selectbest_frame(original_data, new_file_path):
    """
    Creates a new JSON file replicating the structure of the original data,
    but with empty lists for all terminal values.

    :param original_data: Dictionary containing the original data.
    :param new_file_path: Path for the new .json file to be created.
    """
    # Create a new structure with the same keys but with empty lists as terminal values
    empty_data = string_create_deep_empty_structure(original_data)

    # Write this new structure to a new JSON file
    with open(new_file_path, 'w') as file:
        json.dump(empty_data, file, indent=4)

    return empty_data




# put translation into list_skeleton_json

In [40]:
# Helper Function
def populate_skeleton_json_with_data(skeleton_json, source_data):
    """
    Recursively fills the skeleton_json structure with data from the source_data.
    For lists, it merges items instead of replacing them, ensuring unique values.

    :param skeleton_json: The skeleton_json structure with potentially empty lists as terminal values.
    :param source_data: The source data structure from which values are to be copied.
    :return: The skeleton_json structure filled with values from source_data, with care for unique list items.
    """
    if isinstance(skeleton_json, dict) and isinstance(source_data, dict):
        for key in skeleton_json:
            if key in source_data:
                # Recurse into nested dictionaries or replace/merge list and terminal values
                skeleton_json[key] = populate_skeleton_json_with_data(skeleton_json[key], source_data[key])
    elif isinstance(skeleton_json, list):
        # Ensure unique items are added from source_data to skeleton_json list, preserving existing items.
        skeleton_json.extend([item for item in source_data if item not in skeleton_json])
        return skeleton_json
    elif isinstance(skeleton_json, list) and not skeleton_json:
        # For an empty list in skeleton_json, directly copy the source_data list to it
        return source_data.copy()
    # For terminal values, or if the structures don't match (dict/list), return source_data to replace or fill in.
    else:
        return source_data
    return skeleton_json


## save json utility

In [41]:
import os
from datetime import datetime as dt
import json  # Added missing import

# Helper Function
def save_json_to_file(input_text, file_name, target_language, optional_tag=""):
    """
    Saves a JSON object to a file.

    :param data: The JSON object to save.
    :param file_path: The path to the JSON file where the object should be saved.
    """
    # with open(file_path, 'w') as file:
    #     json.dump(data, file, indent=4)

    # make readable time
    date_time = dt.utcnow()
    clean_timestamp = date_time.strftime('%Y%m%d%H%M%S%f')

    new_title = f"{target_language}_{clean_timestamp}_{file_name}"

    new_title = optional_tag + new_title

    # Determine the path to the directory that should contain the file
    directory_path = 'translations'

    # Check if the directory exists
    if not os.path.exists(directory_path):

        # If it does not exist, create it
        # Ensure the directory exists
        try:
            os.makedirs(directory_path, exist_ok=True)  # Ensure the directory is created if it does not exist
        except Exception as e:
            print(f"Error creating directory {directory_path}: {e}")
            return  # Exit the function if directory creation fails

    # Determine the path to the file that should be saved
    file_path = os.path.join(directory_path, new_title)

    # Save the JSON data to the file with UTF-8 encoding
    with open(file_path, 'w', encoding='utf-8') as outfile:
        json.dump(input_text, outfile, indent=4, ensure_ascii=False)

# Swap-Out, Swap-Back (Wax-on, Wax-off)

In [42]:
import sys


def get_swap_in(input_string):
    """
    returns the value of something not in the string
    """
    placeholder_list = [
        ')))***', '^|.|.|', '###$$$', '%&%&%&', "'''(((",
        '+++,,,', '---...', '///000', '111222', '333444',
        '555666', '777888', '999:::', ';;;<<<', '===>>>',
        '???@@@', 'AAABBB', 'CCCDDD', 'EEEFFF', 'GGGHHH',
        'IIIJJJ', 'KKKLLL', 'MMMNNN', 'OOOPPP', 'QQQRRR',
        'SSSTTT', 'UUUVVV', 'WWWXXX', 'YYYZZZ', '[[[sss',
        ']]]^^^', '___```', 'aaabbb', 'cccddd', 'eeefff',
        'ggghhh', 'iiijjj', 'kkklll', 'mmmnnn', 'oooppp',
        'qqqrrr', 'sssttt', 'uuuvvv', 'wwwxxx', 'yyyzzz',
        '{|{{|||', '}~}~}~~~',
        '!', '"', '#', '$', '%', '&', "'", '(',
        ')', '*', '+', ',', '-', '.', '/', '0', '1',
        '2', '3', '4', '5', '6', '7', '8', '9', ':',
        ';', '<', '=', '>', '?', '@', 'A', 'B', 'C',
        'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
        'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U',
        'V', 'W', 'X', 'Y', 'Z', '[', ']', '^',
        '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g',
        'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p',
        'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y',
        'z', '{', '|', '}', '~',
    ]

    for this_placeholder in placeholder_list:
        if this_placeholder not in input_string:
            print(f"item found -> {this_placeholder}")
            return this_placeholder

    return "Unable to find item not in the input_string"


# helper_function
def swap_two(input_string, item_1, item_2):
    """
    Swap two items in a string. Tada!

    or change only the first item, the 2nd item is optional.

    protection from swap-collisions is included
    """

    original_string = input_string

    # validity sanity check (item_2 is optional, only item 1 is needed)
    if item_1 not in input_string:
        print(f"NOTHING DONE: item to match not in string -> {item_1} vs. {input_string}")
        return input_string

    use_this_placeholder = ';;;<<<'

    """all possible ascii placeholders, and more:
    while there is a risk of item 1 or two coliding with the placeholder
    there is also a risk of the placeholder coliding with part of the string
    the longer the string, the more likely it contains any given single
    ascii character
    """
    placeholder_list = [
        ')))***', '^|.|.|', '###$$$', '%&%&%&', ';;;<<<',
        '+++,,,', '---...', '///000', '111222', '333444',
        '555666', '777888', '999:::', ';;;<<<', '===>>>',
        '???@@@', 'AAABBB', 'CCCDDD', 'EEEFFF', 'GGGHHH',
        'IIIJJJ', 'KKKLLL', 'MMMNNN', 'OOOPPP', 'QQQRRR',
        'SSSTTT', 'UUUVVV', 'WWWXXX', 'YYYZZZ', '[[[sss',
        ']]]^^^', '___```', 'aaabbb', 'cccddd', 'eeefff',
        'ggghhh', 'iiijjj', 'kkklll', 'mmmnnn', 'oooppp',
        'qqqrrr', 'sssttt', 'uuuvvv', 'wwwxxx', 'yyyzzz',
        "'''(((", '{|{{|||', '}~}~}~~~',
        '!', '"', '#', '$', '%', '&', "'", '(',
        ')', '*', '+', ',', '-', '.', '/', '0', '1',
        '2', '3', '4', '5', '6', '7', '8', '9', ':',
        ';', '<', '=', '>', '?', '@', 'A', 'B', 'C',
        'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
        'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U',
        'V', 'W', 'X', 'Y', 'Z', '[', ']', '^',
        '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g',
        'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p',
        'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y',
        'z', '{', '|', '}', '~',
    ]

    """
    check that:
    item_2 is not in placeholder
    that placeholder does not collide with something in string
    """
    placeholder_ok = False
    for this_placeholder in placeholder_list:
        print(f"trying -> '{this_placeholder}'")
        if (item_1 not in this_placeholder) and (item_2 not in this_placeholder) and (this_placeholder not in input_string):
            use_this_placeholder = this_placeholder
            placeholder_ok = True
            print(f"use_this_placeholder -> {use_this_placeholder}")
            break
        else:
            print("collision detected, try next placeholder...")

    if not placeholder_ok:
        # print error message and exit program
        message = """FAILED: collision error,
        for swap_two(),
        a new placeholder needed.
        action item: add novel option to placeholder_list
        """
        print(message)
        sys.exit()

    # Replace item_1 with a temporary placeholder
    output_swapped_string = input_string.replace(item_1, use_this_placeholder)
    print(output_swapped_string)

    # Replace item_2 with item_1
    output_swapped_string = output_swapped_string.replace(item_2, item_1)
    print(output_swapped_string)

    # Replace that temporary placeholder with item_2 (done and done)
    output_swapped_string = output_swapped_string.replace(use_this_placeholder, item_2)

    message = f"""
    Final comarison:
    old -> {original_string}
    new -> {output_swapped_string}
    """
    print(message)

    return output_swapped_string


input_string = ""
target = "\'"
swapper = get_swap_in(input_string)

# Run before
swap_two(input_string, target, swapper)

# Run After
swap_two(input_string, target, swapper)

item found -> )))***
NOTHING DONE: item to match not in string -> ' vs. 
NOTHING DONE: item to match not in string -> ' vs. 


''

# Functions to call Mistral

In [43]:
"""
related tools:

api throttle_protection
api async

import requests
"""
import requests
import json
import os
import re
import time

# using google secretes
from google.colab import userdata
mistral_api_key = userdata.get('mistral_api_key')

# Define the endpoint URL
endpoint_url = "https://api.mistral.ai/v1/chat/completions"

# Set the headers
headers = {
  "Content-Type": "application/json",
  "Accept": "application/json",
  "Authorization": f"Bearer {mistral_api_key}"
}

"""
# mode: [{"role": "user", "content": "say yes"}]

    # Define the request body
    request_body = {
      "model": "mistral-small",
      "messages": [{"role": "user", "content": user_input}]
    }

    # Send the request
    response = requests.post(endpoint_url, headers=headers, json=request_body)
"""


# Helper Function
def segment_for_adding_to_context_history(role, comment):

    if role == 'user':
        segment = {"role": "user", "content": comment}

    elif role == 'assistant':
        segment = {"role": "assistant", "content": comment}

    elif role == 'system':
        segment = {"role": "system", "content": comment}

    else:
        print("error segment_for_adding_to_context_history(role, comment)")
        print(role, comment)
        print('error')

    return segment


# Helper Function
def add_to_context_history(user_input, context_history, role):

    if role == 'user':
        segment = {"role": "user", "content": user_input}

    elif role == 'assistant':
        segment = {"role": "assistant", "content": user_input}

    elif role == 'system':
        segment = {"role": "system", "content": user_input}

    else:
        print('error add_to_context_history() error')
        print(role, user_input)
        print('role error?')

    context_history.append( segment )

    return context_history


# Helper Function
def prompt_user(user_input, context_history):

    context_history.append( segment_for_adding_to_context_history("user", user_input) )

    return context_history


# Helper Function
def one_response_to_user(user_input, context_history, use_this_model):
    """
    Input: context_history
    Ouput Tuple: assistant_says, context_history
    """

    counter(3)

    # prompt user
    context_history = prompt_user(user_input, context_history)

    # prompt assistant
    response = ask_mistral_tiny(context_history, use_this_model)

    return response


# Helper Function
def counter(timeout=10):
    count = 0
    start_time = time.time()
    while time.time() - start_time < timeout:
        count += 1
        time.sleep(1)
    return count


# Helper Function
def ask_mistral_tiny(context_history, use_this_model):
    # Define the request body
    request_body = {
      "model": use_this_model,
      "messages": context_history
    }


    # pause if error occurs
    counter(2)

    #################
    #################
    # Hit the ai api
    #################
    #################
    # Send the request
    response = requests.post(endpoint_url, headers=headers, json=request_body)

    # Check the response status code
    if response.status_code != 200:
        print(f"Error: {response.status_code} {response.text}")
        return None

        # pause if error occurs
        counter(10)

    # Get the response data
    response_data = response.json()

    # Print the Mistral response

    ##
    ##
    # Turn this print on to see full return data
    ##
    ##
    """
    e.g.
    {
      "id": "635cb8d445ujhe5546bb64e5e7",
      "object": "chat.completion",
      "created": 170hrjfjf7084,
      "model": "mistral-tiny",
      "choices": [
        {
          "index": 0,
          "message": {
            "role": "assistant",
            "content": "Enjoy your cup of tea!"
          },
          "finish_reason": "stop",
          "logprobs": null
        }
      ],
      "usage": {
        "prompt_tokens": 575,
        "total_tokens": 629,
        "completion_tokens": 54
      }
    }
    """
    # print(json.dumps(response_data, indent=2))
    # print(type(response_data))

    output = response_data
    # print(type(output))
    # print(type(output["choices"][0]))

    # extract just the 'what they said' part out
    assistant_says = output["choices"][0]['message']['content']


    return assistant_says


# functions to call openAI api

In [44]:
# !pip install -q openai

In [45]:
# # functions to call openAI 2024 api


# import openai
# import time
# from openai import AsyncOpenAI

# # my_api_key = ""

# from google.colab import userdata
# my_api_key = userdata.get('open_ai_key')

# from openai import OpenAI

# client = OpenAI(
#     # defaults to os.environ.get("OPENAI_API_KEY")
#     api_key = my_api_key,
# )

# """
# api module parts
# """

# import string
# import asyncio
# from openai import AsyncOpenAI
# import threading

# ############
# # Not Async
# ############
# """
# openai_call_context_timeout() with x-retries and time-limit freeze protection
# """


# def container_api_call(client, model, temp, context, result_container):
#     try:
#         completion = client.chat.completions.create(
#             model=model,
#             temperature=temp,
#             messages=context
#         )
#         result_container['completion'] = completion
#     except Exception as e:
#         result_container['error'] = e


# def run_api_call_with_timeout(client, model, temp, context, timeout):
#     """
#     flags the thread and moves to background after time-out
#     but does not attempt to 'kill' the thread.
#     If thread finishes, output ~should stay in background
#     un-unused by anything.
#     """
#     try:
#         result_container = {}
#         api_thread = threading.Thread(target=container_api_call, args=(client, model, temp, context, result_container))
#         api_thread.start()
#         api_thread.join(timeout)

#         if api_thread.is_alive():
#             # If the API call is still running after the timeout
#             print("The API call timed out.")
#             return None  # Or handle the timeout situation as needed

#         if 'error' in result_container:
#             # If there was an error during the API call
#             print("An error occurred:", result_container['error'])
#             return None

#         return result_container.get('completion')

#     except Exception as e:
#         raise( e, f"ERROR in function: run_api_call_with_timeout(input_string)" )


# # helper function ()
# def openai_call_context_timeout(client, context, model="gpt-4", max_retries=10, temp=0.9, timeout_min=8):
#     """
#     requires: container_api_call(), run_api_call_with_timeout()
#     """
#     # load_dotenv()
#     # openai.api_key = os.getenv("OPENAI_API_KEY")

#     # convert timeout into seconds (int)
#     timeout_sec = int(timeout_min * 60)

#     # makes sure number is not zero
#     if timeout_sec <= 0:
#         timeout_sec = 1

#     """
#     Time parameters for retry to double in duration until the max seconds limit
#     """
#     base_delay_sec = 2
#     max_delay_sec = 16

#     retries = 0
#     while retries <= max_retries:
#         try:
#             # call API within a time-out filter
#             completion = run_api_call_with_timeout(
#                 client,
#                 model=model,
#                 temp=temp,
#                 context=context,
#                 timeout=timeout_sec
#             )

#             text = completion.choices[0].message.content

#             print(f"openai_call_context_timeout() -> {text}")

#             return text

#         except Exception as e:
#             print(f"an error occurred: {e}")
#             retries += 1
#             if retries > max_retries:
#                 print("max retries reached. exiting.")
#                 raise

#             # using min() to ensure that delay <= max
#             real_delay_sec = min(max_delay_sec, (2 ** retries) * base_delay_sec)
#             print(f"retrying in {real_delay_sec} seconds...")
#             # pause before retrying
#             time.sleep(real_delay_sec)
#             """
#             Note: time delay is not a crash-exception, it is a re-try
#             """




In [46]:
# # test
# context = []

# # "system"
# system_prompt = f"""You are a talking bird."""

# context.append({
#     "role": "system", "content": system_prompt
# })

# prompt = "Squawk, I am a bird."
# context.append({
#     "role": "assistant", "content": prompt
# })

# prompt = "what is a bird?"
# context.append({
#     "role": "user", "content": prompt
# })

# print(context)

# # result = openai_call_context(context, model="gpt-4", max_retries=2)

# # result = await openai_call_context_timeout_async(client, context, model="gpt-3.5-turbo", max_retries=2, temp=0.9, timeout_min=2)

# # print(f"result -> {result}")


In [47]:

# client = OpenAI(
#     # defaults to os.environ.get("OPENAI_API_KEY")
#     api_key = my_api_key,
# )


# pause_time_seconds = .5

# # Helper Function
# def pause_ok():
#     time.sleep(pause_time_seconds)
#     return "Pause OK!"

# # Helper Function: Call API Function
# def call_openai_chat_api(this_input, select_model=3):
#     """
#     Requires:
#       import time
#       pip install openai
#       import openai
#       api key from openai, included in subscription:
#       https://platform.openai.com/account/api-keys

#       # Helper Function
#       def pause_ok():
#           time.sleep(pause_time_seconds)
#           return "OK!"


#     Chat type query to openAI:
#     pecify Model, default is "gpt-3.5-turbo"
#     user input-2 selects which model to use: 3, 4

#     Sample use:

#     this_input = "Hello."
#     select_model = 3

#     call_openai_chat_api(this_input, select_model=3)
#     """

#     try:
#         if select_model == 4:
#             this_model = "gpt-4"

#         elif select_model == "gpt-4":
#             this_model = "gpt-4"

#         elif select_model == "gpt-3.5-turbo":
#             this_model = "gpt-3.5-turbo"

#         elif select_model == 3:
#             this_model = "gpt-3.5-turbo"

#         # Pause to avoid throttling
#         print(f"Pausing {pause_time_seconds} seconds...")
#         print(pause_ok())
#         # time.sleep(pause_time_seconds)
#         # print(f"Pausing {pause_time_seconds} seconds...OK!")


#         ## Hit API
#         completion = client.chat.completions.create(model=this_model, messages=[{"role": "user", "content": this_input}])

#         # Terminal Inspection for Examination (optional)
#         # print(completion)

#         ## Results (slicing into the json object output)
#         results = completion.choices[0].message.content

#     except Exception as e:
#         print("API call function call_openai_chat_api(this_input, select_model=3) failed, error = ", e)
#         return e

#     return results


In [48]:
# call_openai_chat_api("testing")

## Add System instructions for rules

In [49]:
# Helper Function
def set_translator__system_prompt(context_history, target_language):

    ################
    # System Prompt
    ################

    example_1 = "a happy cat"

    example_2 = {
        "translation": "chat heureux"
    }

    example_bad = {
      'NOT THIS': 'NO SINGLE QUOTES'
      }


    example_3 = {
        "translation": "S'inscrire"
    }



    # set translation language and structure of output in system
    text_input = f"""
    You are an expert helpful {target_language} language translator bot that produces high
    quality professional translations in precise json formats.

    You translate writen language, not emojis or syntax not
    readable by a person. You use normal capitalization,
    not all upper case. You use normal full words, not single letters
    or obscure abreviations.

    You always deliver your translation in the same correct json format.
    starting with ```json and ending with ```.
    "translation": "YOUR TRANSLATION HERE"

    e.g. If the original phrase is:
    {example_1}
    Then your translation format is like this, with no other commentary needed:
    ```jsons
    {example_2}
    ```

    If the target language is french, and the translation is S'inscrire:
    You output
    {example_3}


    Your translation is always expressed using valid json syntax,
    using double quotes only in json.
    (e.g. no trailing delimiter, escape conflicting characters, etc).


    You only translate into {target_language}.
    You only produce json output in exactly this structure
    ```jsons
    {example_2}
    ```
    Your translations are clear, accurate, helpful, honrable, brief, polite, and professional.
    Your do you best to tranlsate every leaf value field leaving nothing blank.
    Every final leaf values MUST be translated.

    You always double check your work and make sure the translation is
    excellent in the context of the whole body of translation.
    """
    role = "system"

    context_history.append( segment_for_adding_to_context_history(role, text_input) )

    # # inspection
    # print("set_translator__system_prompt -> ", context_history)

    return context_history


## Add 'user' request for translation

In [50]:
# Helper Function
def set_translate__user_prompt(context_history, target_language, original_data):

    ###########################
    # User Translation Request
    ###########################

    example_1 = "a happy cat"

    example_2 = {
        "translation": "your translation here"
    }

    example_bad = {
      'NOT THIS': 'NO SINGLE QUOTES'
      }

    example_bad2 = """{\'translation\': "BAD!! NO SINGLE QUOTES"}"""


    # set translation language and structure of output in system
    text_input = f"""

    The person we are trying to help needs text string
    translated into the {target_language} language.

    Carefully translate this original text string into {target_language}.
    The original string is: {original_data}

    Double check your work and make sure the translation is
    accurate, brief, and polite.

    Make sure your translation is expressed using valid json syntax.
    Always use double quotes " only around items in json.

    Formatting for json must be like this in double-quotes for jsone.g.
    ```json
    {example_2}
    ```

    You can do it!!
    """
    role = "user"

    context_history.append( segment_for_adding_to_context_history(role, text_input) )

    # # inspection
    # print("set_translate__user_prompt", context_history)

    return context_history



# json inspection

In [51]:
# Helper Function
def dict_leaf_detection_boolean_true_means_defective(input_dict):
    """
    Recursively checks if any terminal leaf node in a nested structure is an empty string,
    an empty list, or a single character.

    :param input_dict: The nested input_dict structure to check.
    :return: True if any leaf node is an empty string, an empty list, or a single character; False otherwise.
    """
    if isinstance(input_dict, dict):  # If the current item is a dictionary
        return any(dict_leaf_detection_boolean_true_means_defective(value) for value in input_dict.values())
    elif isinstance(input_dict, list):  # If the current item is a list
        return any(dict_leaf_detection_boolean_true_means_defective(item) for item in input_dict)
    else:  # If the current item is a leaf node
        return input_dict == "" or (isinstance(input_dict, list) and len(input_dict) == 0) or (isinstance(input_dict, str) and len(input_dict) == 1)


In [52]:
import json
import re

def clean_and_convert_to_json(input_str):
    # Step 1: Automatically handle the known structure without using regex
    # This involves replacing the problematic starting and ending quotes if they exist.
    cleaned_str = input_str.replace('\'{', '{').replace('}\'', '}')

    # Step 2: Use regex to replace improperly escaped single quotes with double quotes,
    # while ensuring not to change single quotes within words.
    # This regex targets single quotes that are either at the start of the string or
    # followed by a colon or a space, which we assume are not part of the actual text.
    cleaned_str = re.sub(r"(?<=\{|\,)\s*'([^']+)'(?=\:)", r'"\1"', cleaned_str)

    # Additional handling to replace the escaped single quotes inside the value
    cleaned_str = cleaned_str.replace("\\'", "'")

    # Step 3: Convert the cleaned string to a JSON object
    # try:
    #     json_obj = json.loads(cleaned_str)
    # except json.JSONDecodeError as e:
    #     print(f"Error decoding JSON: {e}")
    #     return None

    return cleaned_str

# Example input
input_str = '\'{"translation": "S\\\'inscrier"}\''
json_obj = clean_and_convert_to_json(input_str)

print(json_obj)


{"translation": "S'inscrier"}


In [53]:
# Helper Function
def check_function_description_keys(dict_str):
    """
    This function CAN fail and should fail
    if the AI needs to retry at a task.
    Do not stop server when this this triggers an exception.

    edge case: before there is a populated output_log

    if passing, this function will return a valid json object
    """

    """
    Extracts JSON string enclosed between ```json and ``` markers.

    Parameters:
    - text (str): The input text containing the JSON block.

    Returns:
    - str: The extracted JSON string, or an empty string if no JSON block is found.
    """
    print(f"\n\n Starting check_function_description_keys, dict_str -> {repr(dict_str)} {type(dict_str)}")

    # input("breakpoint")



    ########################
    # Check Json Formatting
    ########################


    # # pre-check
    #     # load
    # try:
    #     if "'" not in dict_str:

    #         # Load the string into a Python dictionary
    #         dict_data = json.loads(dict_str)

    #         dict_str = dict_data['translation']

    #         if dict_leaf_detection_boolean_true_means_defective(dict_str):
    #             return dict_str

    #         else:
    #             print(f"Failed dict_str precheck")

    # except:
    #     print(f"Failed dict_str precheck")




    # extraction 1
    try:
        if """```json""" in dict_str:


            pattern = r'```json\n([\s\S]*?)\n```'
            match = re.search(pattern, dict_str)
            dict_str =  match.group(1) if match else ''

    except Exception as e:
        print(f"\nTRY AGAIN: check_function_description_keys() extraction from markdown failed: {e}")
        print(f"Failed dict_str -> {repr(dict_str)}")
        return False

    print(f"\n  extraction-1 from markdown dict_str -> {repr(dict_str)} {type(dict_str)}")

    # ```json\n{\'translation\': "S\'inscrier"}\n```'

    try:
        # if ("""{'translation': '""" in dict_str) and ("""'}""" in dict_str):
        #     dict_str.replace( """{'translation':""", """{"translation":"""  )
        #     dict_str.replace( """'}""", """"}"""  )

        # if ("""{\'translation\': \'""" in dict_str) and ("""\'}""" in dict_str):
        #     dict_str.replace( """{\'translation\':""", """{"translation":"""  )
        #     dict_str.replace( """\'}""", """"}"""  )

        if ("""{\'translation\':""" in dict_str):
            dict_str = dict_str.replace( """{\'translation\':""", """{"translation":"""  )

        if ("""{'translation':""" in dict_str):
            dict_str = dict_str.replace( """{'translation':""", """{"translation":"""  )

        if ("""{\\'translation\\':""" in dict_str):
            dict_str = dict_str.replace( """{\\'translation\\':""", """{"translation":"""  )

    except Exception as e:
        print(f"Failed dict_str -> {repr(dict_str)}")
        return False

    print(f" dict_str -> {repr(dict_str)} {type(dict_str)}")


    dict_str = clean_and_convert_to_json(dict_str)
    print(f" clean_and_convert_to_json dict_str -> {repr(dict_str)} {type(dict_str)}")


    try:
        # if ("""{'translation': '""" in dict_str) and ("""'}""" in dict_str):
        #     dict_str.replace( """{'translation':""", """{"translation":"""  )
        #     dict_str.replace( """'}""", """"}"""  )

        # if ("""{\'translation\': \'""" in dict_str) and ("""\'}""" in dict_str):
        #     dict_str.replace( """{\'translation\':""", """{"translation":"""  )
        #     dict_str.replace( """\'}""", """"}"""  )

        if ("""{\'translation\':""" in dict_str):
            dict_str = dict_str.replace( """{\'translation\':""", """{"translation":"""  )

        if ("""{'translation':""" in dict_str):
            dict_str = dict_str.replace( """{'translation':""", """{"translation":"""  )

        if ("""{\\'translation\\':""" in dict_str):
            dict_str = dict_str.replace( """{\\'translation\\':""", """{"translation":"""  )

    except Exception as e:
        print(f"Failed dict_str -> {repr(dict_str)}")
        return False



    # clean
    try:
        """
        Swap in and swap out escaped single commas
        to avoid them being removed during reformatting
        or the reformatting otherwise breaking the json
        """


        # if ("\'" in dict_str) or ("""\\'""" in dict_str):
        #     print("escaped single quote found")

        #     input_string = dict_str
        #     target = "\'"
        #     swapper = get_swap_in(input_string)

        #     # Run before
        #     swap_two(input_string, target, swapper)

        #     # # This conflicted with free language in description section...
        #     dict_str = dict_str.replace("'", '"')

        #     # Run After
        #     swap_two(input_string, target, swapper)


        # else:
        #     # # This conflicted with free language in description section...
        #     dict_str = dict_str.replace("'", '"')


        # try safety cleaning
        dict_str = dict_str.replace("True", "true")
        dict_str = dict_str.replace("False", "false")
        dict_str = dict_str.replace("None", "null")


        # remove trailing delimiter comma
        print(f"{dict_str[:-6]}")
        dict_str = dict_str.replace('",\n}', '"\n}')

    except Exception as e:
        print(f"\nTRY AGAIN:try safety cleaning: {e}")
        print(f"Failed repr(dict_str) -> {repr(dict_str)}")
        return False

    # load
    try:
        # try converting
        print(f"dict_str -> {repr(dict_str)} {type(dict_str)}")

        # Load the string into a Python dictionary
        dict_data = json.loads(dict_str)

    except Exception as e:
        print(f"\nTRY AGAIN: trying json.loads(dict_str) Dictionary load failed: {e}")
        print(f"Failed repr(dict_str) -> {repr(dict_str)}")
        return False


    # extraction 2
    try:
        # Extract the value associated with the key 'translation'
        dict_str = dict_data['translation']

    except Exception as e:
        print(f"\nTRY AGAIN: check_function_description_keys() extraction 2 from translation = dict_data['translation'] failed: {e}")
        print(f"Failed repr(dict_str) -> {repr(dict_str)}")
        return False


    # try:
    #     # if test fails
    #     if dict_leaf_detection_boolean_true_means_defective(dict_str):
    #         return False

    # except Exception as e:
    #     print(f"\nTRY AGAIN: dict_leaf_detection_boolean_true_means_defective() empty or stub leaf found: {e}")
    #     print(f"Failed dict_str -> {dict_str}")
    #     return False


    print(f"\n  final extracted from markdown, dict, etc. ->{repr(dict_str)}")

    # if ok...
    return dict_str



# Call api within: Check json structure against original

In [54]:
# helper function
def call_api_within_structure_check(context_history, use_this_model, mode_locale, skeleton_json):
    retry_counter = 0
    json_ok_flag = False

    # see
    mistal_model_list = [
        "mistral-tiny",
        "mistral-small",
        "mistral-large-latest",
    ]

    # see https://platform.openai.com/docs/guides/text-generation
    open_ai_model_list = [
        "gpt-4",
        "gpt-4-turbo-preview",
        "gpt-3.5-turbo"
    ]


    gguf_model_list = [
        "jais",
        "tiny_llama",
        "mistral8x7...",
        ""
    ]

    while not json_ok_flag:

        ####################
        # get a translation
        ####################

        try:
            # check json structure

            ########################
            # Select Model and Mode
            ########################

            # for off-line local mode
            if mode_locale == "gguf":
                """
                TODO: code for using local gguf model
                """
                pass

            ################
            # for cloud api
            ################
            elif use_this_model in mistal_model_list:
                print(f"Mistral api selected...{use_this_model}")
                dict_str = ask_mistral_tiny(context_history, use_this_model)

            elif use_this_model in open_ai_model_list:
                print(f"openAI api selected...{use_this_model}")
                dict_str = openai_call_context_timeout(client, context_history, model=use_this_model, max_retries=10, temp=0.9, timeout_min=8)

            else:
                print(f"no known api selected...{use_this_model}")
                raise f"No known model option chosen...use_this_model -> {use_this_model}"


        except Exception as e:
            jsonchecked_translation = None
            print(f"Failed: {str(e)}")

        jsonchecked_translation = check_function_description_keys(dict_str)

        if jsonchecked_translation:
            json_ok_flag = True

        else:
            retry_counter += 1
            print(f"retry_counter -> {retry_counter}")

            # # breakpoint
            # input("Breakpoint")

    print(f"retry_counter -> {retry_counter}")

    return jsonchecked_translation


# Crawler model call

In [55]:
# helper function
def crawlwer_call_api_within_json_structure_check(context_history, use_this_model, mode_locale, skeleton_json):
    retry_counter = 0
    json_ok_flag = False

    # see
    mistal_model_list = [
        "mistral-tiny",
        "mistral-small",
        "mistral-large-latest",
    ]

    # see https://platform.openai.com/docs/guides/text-generation
    open_ai_model_list = [
        "gpt-4",
        "gpt-4-turbo-preview",
        "gpt-3.5-turbo"
    ]


    gguf_model_list = [
        "jais",
        "tiny_llama",
        "mistral8x7...",
        ""
    ]

    while not json_ok_flag:

        ####################
        # get a translation
        ####################

        try:
            # check json structure

            ########################
            # Select Model and Mode
            ########################

            # for off-line local mode
            if mode_locale == "gguf":
                """
                TODO: code for using local gguf model
                """
                pass

            ################
            # for cloud api
            ################
            elif use_this_model in mistal_model_list:
                print(f"Mistral api selected...{use_this_model}")
                dict_str = ask_mistral_tiny(context_history, use_this_model)

            elif use_this_model in open_ai_model_list:
                print(f"openAI api selected...{use_this_model}")
                dict_str = openai_call_context_timeout(client, context_history, model=use_this_model, max_retries=10, temp=0.9, timeout_min=8)

            else:
                print(f"no known api selected...{use_this_model}")
                raise f"No known model option chosen...use_this_model -> {use_this_model}"


        except Exception as e:
            jsonchecked_translation = None
            print(f"Failed: {str(e)}")

        jsonchecked_translation = check_function_description_keys(dict_str, skeleton_json)

        if jsonchecked_translation:
            json_ok_flag = True

        else:
            retry_counter += 1

    print(f"retry_counter -> {retry_counter}")

    return jsonchecked_translation


# Put Flesh on the Skeleton
- add translations to the lists

In [56]:
# # helper function
# def set_select_best__system_prompt(context_history, target_language):

#     ########################################
#     # System Prompt to pick best traslation
#     ########################################

#     example_2 = {
#         "translation": "your translation here"
#     }

#     example_bad = {
#       'NOT THIS': 'NO SINGLE QUOTES'
#       }

#     example_bad2 = """{\'translation\': "BAD!! NO SINGLE QUOTES"}"""

#     # set translation language and structure of output in system
#     text_input = f"""
#     You are a helpful expert translator bot that selects high
#     quality professional translations from a list and
#     selects the best translation.

#     You always present your selection in the same precise json format:
#     {example_2}

#     NOT like this with single quotes {example_bad}
#     You never ever write like this: {example_bad2}
#     You never use single quotes around items in a json. EVER!!

#     Your final translation selections are clear, accurate, helpful, honrable, brief, polite, and professional.
#     Always select one best translation, only one.

#     You always do your best to produce top quality results.
#     """
#     role = "system"

#     context_history.append( segment_for_adding_to_context_history(role, text_input) )

#     return context_history



In [57]:
# helper function
def set_select_best__system_prompt(context_history, target_language):

    ########################################
    # System Prompt to pick best traslation
    ########################################

    example_2 = {
        "translation": "your translation here"
    }

    # set translation language and structure of output in system
    text_input = f"""
    You are a helpful expert translator bot that selects high
    quality professional translations from a list and
    selects the best translation.

    You always present your selection in the same precise json format:
    {example_2}

    You always only use double-quotes around items a json, that is extremely important.

    Your final translation selections are clear, accurate, helpful, honrable, brief, polite, and professional.
    Always select one best translation, only one.

    You always do your best to produce top quality results.
    """
    role = "system"

    context_history.append( segment_for_adding_to_context_history(role, text_input) )

    return context_history



In [58]:
# helper function
def set_select_best__user_prompt(context_history, target_language, populated_skeleton, original_data):

    ###########################
    # User Translation Request to select best translation
    ###########################

    example_2 = {
        "translation": "your translation here"
    }


    example_2 = {
        "translation": "your translation here"
    }


    example_bad = {
      'NOT THIS': 'NO SINGLE QUOTES'
      }

    example_bad2 = """{\'translation\': "BAD!! NO SINGLE QUOTES"}"""


    # set translation language and structure of output in system
    text_input = f"""
    Carefully select the best {target_language} language translation
    of {original_data} from this list of translations: {populated_skeleton}.

    Double check your work and make sure the translation is
    accurate, brief, and polite.

    Make sure your translation is expressed using valid json syntax,
    always using double quotes " around items in a json:
    ```json
    {example_2}
    ```

    The structure of your translation must fit that. structure

    Do your best!
    """
    role = "user"

    context_history.append( segment_for_adding_to_context_history(role, text_input) )

    # print("\n\n\n\nset_select_best__user_prompt() context_history", context_history)

    return context_history


# Crawler functions
1. make list of paths
2. extrac path value
3. write path value

In [59]:
"""
####################
# Crawler functions
####################

1. make list of paths   generate_paths(json_object)
2. extrac path value    extract_values_by_paths(json_object, paths_list)
3. write path value     insert_values_by_paths(skeleton_json, paths_list, translated_values)

"""


def generate_paths(json_object, current_path=None):
    if current_path is None:
        current_path = []
    paths_list = []
    if isinstance(json_object, dict):
        for key, value in json_object.items():
            paths_list += generate_paths(value, current_path + [key])
    elif isinstance(json_object, list):
        for idx, item in enumerate(json_object):
            paths_list += generate_paths(item, current_path + [str(idx)])
    else:  # Terminal leaf found
        paths_list.append(current_path)
    return paths_list


def extract_value_by_path(json_object, this_path):
    """
    Extracts a single value from a JSON object based on a single this_path.

    :param json_object: The JSON object from which to extract the value.
    :param this_path: The this_path to the value as a list of keys/indexes.
    :return: The value at the specified this_path.
    """
    for step in this_path:
        if isinstance(step, str) and step.isdigit():
            step = int(step)  # Convert to integer if it's a list index
        json_object = json_object[step]
    return json_object


# def extract_values_by_paths(json_object, paths_list):
#     values = []
#     for path in paths_list:
#         value = json_object
#         for step in path:
#             if step.isdigit():
#                 step = int(step)  # Convert to integer if it's a list index
#             value = value[step]
#         values.append(value)
#     return values


# def insert_values_by_paths(skeleton_json, paths_list, translated_values):
#     for path, translated_value in zip(paths_list, translated_values):
#         target = skeleton_json
#         for step in path[:-1]:
#             step = int(step) if step.isdigit() else step
#             target = target[step]
#         final_step = path[-1]
#         final_step = int(final_step) if final_step.isdigit() else final_step
#         target[final_step] = translated_value


def insert_value_by_path(skeleton_json, this_path, translated_value):
    target = skeleton_json
    for step in this_path[:-1]:  # Navigate through the path except for the last step
        step = int(step) if step.isdigit() else step
        target = target[step]
    final_step = this_path[-1]
    final_step = int(final_step) if final_step.isdigit() else final_step
    target[final_step] = translated_value
    return skeleton_json  # Return the modified object explicitly


def append_value_by_path(json_structure, path, new_value):
    """
    Attempts to append a value to a list at the specified path within the json_structure.
    Provides more detailed error messages if the path is not found.

    :param json_structure: The JSON structure to update.
    :param path: The path to the target list as a list of keys.
    :param new_value: The value to append.
    """
    target = json_structure
    for i, step in enumerate(path):
        try:
            if isinstance(target, dict) and step in target:
                if i == len(path) - 1:  # If it's the final step, append new_value
                    if isinstance(target[step], list):
                        target[step].append(new_value)
                        return json_structure  # Return updated structure
                    else:
                        raise ValueError(f"Target at path {'/'.join(path)} is not a list.")
                else:  # Not the final step, move deeper into the structure
                    target = target[step]
            else:
                raise ValueError(f"Path error: {'/'.join(path[:i+1])} not found.")
        except TypeError as e:
            raise TypeError(f"TypeError accessing {'/'.join(path[:i])}: {str(e)}")

    # If for some reason the loop completes without appending (shouldn't happen if errors are caught)
    raise ValueError("Failed to append the value: Path processing error.")


# def append_value_by_path(json_structure, path, new_value):
#     """
#     Appends a value to a list at the specified path within the json_structure.

#     :param json_structure: The JSON structure (dictionary) to update.
#     :param path: A list representing the path to the target list. Each element in the path
#                  is a dictionary key.
#     :param new_value: The new value to append to the list at the target path.
#     """
#     target = json_structure
#     # Navigate through the path except for the last step
#     for step in path[:-1]:
#         if isinstance(target, dict) and step in target:
#             target = target[step]
#         else:
#             raise ValueError(f"Path error: {step} not found.")

#     final_step = path[-1]
#     if isinstance(target, dict) and final_step in target and isinstance(target[final_step], list):
#         target[final_step].append(new_value)
#     else:
#         raise ValueError(f"Final step error: {final_step} not found or not a list.")


def replace_leaf_by_path(json_structure, path, new_value):
    """
    Replaces the value at the specified path within the json_structure with new_value.

    :param json_structure: The JSON structure (dictionary) to update.
    :param path: A list representing the path to the target leaf. Each element in the path
                 is a dictionary key or a list index.
    :param new_value: The new value to replace at the target leaf.
    """
    target = json_structure
    # Iterate through the path to find the target container (dict or list) for the new value
    for step in path[:-1]:  # Navigate through the path except for the last step
        if step.isdigit():  # Convert to int if the step is an index in a list
            step = int(step)
        target = target[step]

    final_step = path[-1]
    if final_step.isdigit():  # Convert the last step if it's an index
        final_step = int(final_step)

    # Replace the old value with the new value at the final step
    target[final_step] = new_value


# Main Function

In [60]:
# Main Function

def translate_json(list_of_targeted_languages, use_this_model, mode_locale, number_of_preliminary_translations):

    ######################
    # Translation Factory
    ######################

    # Example usage
    json_files_list = list_json_files_in_cwd()

    if not json_files_list:
        print("Error: You missed a step, No Json files were provided.")
        raise "No Json files were provided."

    # inspection
    print("JSON files in the CWD:", json_files_list)

    for this_original_json_file in json_files_list:

        # Load the original JSON file
        original_data = load_json_file(this_original_json_file)

        # Create a new JSON file with the deep empty structure
        name_of_skeleton_saved_file = 'empty_lists_' + '_' +this_original_json_file

        skeleton_json = create_empty_json_file(original_data, name_of_skeleton_saved_file)

        name_of_EMPTY_select_best_frame_saved_file = 'empty_string_best_' + '_' +this_original_json_file
        select_best_frame = create_empty_selectbest_frame(original_data, name_of_EMPTY_select_best_frame_saved_file)


        #########################################
        # Crawler: Make preliminary Translations
        #########################################

        paths_list = generate_paths(original_data)
        check_paths_list = generate_paths(skeleton_json)
        select_best_frame_paths_list = generate_paths(select_best_frame)

        print(f"""
        paths_list                   {paths_list}
        check_paths_list             {check_paths_list}
        select_best_frame_paths_list {select_best_frame_paths_list}
        """)


        # Sanity check
        if paths_list != select_best_frame_paths_list:
            error_message = "Error: Path lists between the original JSON and its skeleton do not match."
            print(error_message)
            raise ValueError(error_message)


        # for this language
        for target_language in list_of_targeted_languages:

            # make a blank frame of lists for the translations
            populated_skeleton = skeleton_json

            # for this leaf
            for this_path in paths_list:

                untranslated_leaf = extract_value_by_path(original_data, this_path)

                # breakpoint
                print(f"\n\n breakpoint 5: untranslated_leaf -> {untranslated_leaf}")
                # input("breakpoint")

                # make empty conversation
                # reset context history for new 'conversation' about translation
                context_history = []


                # Set Prompts per one language
                # System Instructions
                context_history = set_translator__system_prompt(context_history, target_language)
                # User Prompt
                context_history = set_translate__user_prompt(context_history, target_language, untranslated_leaf)

                # breakpoint
                print(f"\n\n breakpoint 5: context_history -> {context_history}")
                # input("breakpoint")


                # making N translation-versions
                for i in range(number_of_preliminary_translations):
                    """
                    using both the populated skeleton and the original file:
                    - tree-search through both (original and blank-list-skeleton) in the same way
                    - check and guarantee that the dict-address (often nested)
                      is the same for the original and where the answers are recorded
                    - part 1: extract just the next terminal leaf, return this.
                    separate step next ->
                    - part 2: put a new (language translated value) in the corresponding
                    place in blank-skeleton list.
                    """
                    """
                    ####################
                    # Crawler functions
                    ####################

                    1. make list of paths   generate_paths(json_object)
                    2. extrac path value    extract_value_by_path(original_data, this_path)
                     (translate)
                    3. add to list          append_value_by_path(json_structure, path, new_value)
                    4. write final value    insert_value_by_path(skeleton_json, paths_list, translated_values)

                    """

                    ############
                    # Translate
                    ############
                    translated_value = call_api_within_structure_check(context_history, use_this_model, mode_locale, skeleton_json)

                    # add-insert value to json
                    print(f"Before appending: {skeleton_json}")
                    print(f"this_path -> {this_path}")
                    skeleton_json = append_value_by_path(skeleton_json, this_path, translated_value)


                #####################################################
                # Select Top Top Goodest Translation Star-Good-Prime
                #####################################################

                save_json_to_file(populated_skeleton, this_original_json_file, target_language, "set_of_translations_")

                # reset context history for new 'conversation' about selection
                context_history = []

                print("\n\n\nSelect Top Top Goodest Translation Star-Good-Prime")
                # # inspection breakpoint
                # print(f"\n\n breakpoint 5: populated_skeleton -> {populated_skeleton}")
                # # input("breakpoint")

                # set prompts to select best translation
                list_of_options = extract_value_by_path(skeleton_json, this_path)

                # System Instructions
                context_history = set_select_best__system_prompt(context_history, target_language)
                # User Prompt
                context_history = set_select_best__user_prompt(context_history, target_language, list_of_options, untranslated_leaf)


                #################
                #################
                # Select Bestest
                #################
                #################
                selected_bestest_value = call_api_within_structure_check(context_history, use_this_model, mode_locale, skeleton_json)


                # add value to json
                select_best_frame = insert_value_by_path(select_best_frame, this_path, selected_bestest_value)

            ##########################
            # per language: save file
            ##########################
            print("trying to save file...")

            # try:
            #     # if test fails
            #     if dict_leaf_detection_boolean_true_means_defective(select_best_frame):
            #         return False

            # except Exception as e:
            #     print(f"\nTRY AGAIN: dict_leaf_detection_boolean_true_means_defective() empty or stub leaf found: {e}")
            #     print(f"Failed dict_str -> {select_best_frame}")
            #     return False

            # add value to json
            save_json_to_file(select_best_frame, this_original_json_file, target_language, "selected_")



# Tranlate Json Files
- Set your language list
- Pick your model
- upload your json files
- Run
- Download / Get your translations (and inspection files if needed)

In [61]:
##########################################################
# https://platform.openai.com/docs/guides/text-generation
##########################################################
"""
gpt-4

gpt-4-turbo-preview

gpt-3.5-turbo
"""

use_this_model = "gpt-4"
use_this_model = "gpt-4-turbo-preview"
use_this_model = "gpt-3.5-turbo"

In [62]:
######################
# Mixtral 8x7 "Small"
######################
use_this_model = "mistral-small"

######################
# Mixtral Large ???
######################
use_this_model = "mistral-large-latest"

####################
# Mistral 7b "Tiny"
####################
use_this_model = "mistral-tiny"

In [63]:
################################################
# Local Off-line Mode, or Online Cloud-api Mode
################################################

"""
"cloud_api"

"gguf"
"""

mode_locale = "cloud_api"

# Choices:

In [64]:
# use_this_model = "gpt-3.5-turbo"
# use_this_model = "mistral-large-latest"

use_this_model = "mistral-tiny"
# use_this_model = "mistral-small"

In [65]:
mode_locale = "cloud_api"

In [66]:
# list_of_targeted_languages = ["Italian", "Spanish", "German", "Czech", "Arabic", "Hindi", "Portuguese", "French"]

In [67]:
# list_of_targeted_languages = ["Italian", "Spanish", "German", "Czech", "Arabic", "Portuguese", "French", "Brazilian Portuguese"]

In [68]:
list_of_targeted_languages = ["Portuguese", "French", "Brazilian Portuguese", "Czech", "Arabic"]

In [69]:
list_of_targeted_languages = ["German", "French"]

In [70]:
# list_of_targeted_languages = ["French"]

In [71]:
number_of_preliminary_translations = 2

In [72]:
translate_json(list_of_targeted_languages, use_this_model, mode_locale, number_of_preliminary_translations)

JSON files in the CWD: ['small_json_4.json']

        paths_list                   [['signUp', 'SignUp']] 
        check_paths_list             [] 
        select_best_frame_paths_list [['signUp', 'SignUp']] 
        


 breakpoint 5: untranslated_leaf -> Sign Up


 breakpoint 5: context_history -> [{'role': 'system', 'content': '\n    You are an expert helpful German language translator bot that produces high\n    quality professional translations in precise json formats.\n\n    You translate writen language, not emojis or syntax not\n    readable by a person. You use normal capitalization,\n    not all upper case. You use normal full words, not single letters\n    or obscure abreviations.\n\n    You always deliver your translation in the same correct json format.\n    starting with ```json and ending with ```. \n    "translation": "YOUR TRANSLATION HERE"\n\n    e.g. If the original phrase is:\n    a happy cat\n    Then your translation format is like this, with no other commentary ne

In [73]:
import json
import re

def clean_and_convert_to_json(input_str):
    # Correcting the input string by replacing single quotes with double quotes, while
    # ensuring not to alter single quotes that are part of the actual content.

    # Since the first word can be handled automatically, as mentioned, we focus on
    # the rest of the string. We're assuming the structure is always like:
    # {'key': 'value'} and converting it to {"key": "value"} format.

    # Step 1: Safely replace the outermost single quotes if they're at the start and end
    if input_str.startswith("'") and input_str.endswith("'"):
        input_str = input_str[1:-1]

    # Step 2: Replace ': " with correct JSON format: ": "
    # This step aims to correct the dictionary string to JSON format without using regex.
    input_str = input_str.replace('\': "', '": "').replace("\': \'", '": "')

    # Step 3: Handle escaped single quotes within values by unescaping them
    # We do not need to escape single quotes in JSON values.
    input_str = input_str.replace("\\'", "'")

    # Convert to JSON
    try:
        json_obj = json.loads(input_str)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None

    return json_obj

# Example input strings
input_str1 = '{\'translation\': "S\'inscrier"}'
input_str2 = '{"translation": "S\'inscrier"}'

# Attempt to clean and convert both strings
json_obj1 = clean_and_convert_to_json(input_str1)
json_obj2 = clean_and_convert_to_json(input_str2)

print(json_obj1)
print(json_obj2)


Error decoding JSON: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
None
{'translation': "S'inscrier"}


In [74]:
import json
import re

def clean_and_convert_to_json(input_str):
    # Step 1: Automatically handle the known structure without using regex
    # This involves replacing the problematic starting and ending quotes if they exist.
    cleaned_str = input_str.replace('\'{', '{').replace('}\'', '}')

    # Step 2: Use regex to replace improperly escaped single quotes with double quotes,
    # while ensuring not to change single quotes within words.
    # This regex targets single quotes that are either at the start of the string or
    # followed by a colon or a space, which we assume are not part of the actual text.
    cleaned_str = re.sub(r"(?<=\{|\,)\s*'([^']+)'(?=\:)", r'"\1"', cleaned_str)

    # Additional handling to replace the escaped single quotes inside the value
    cleaned_str = cleaned_str.replace("\\'", "'")

    # Step 3: Convert the cleaned string to a JSON object
    try:
        json_obj = json.loads(cleaned_str)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None

    return json_obj

# Example input
input_str = '\'{"translation": "S\\\'inscrier"}\''
json_obj = clean_and_convert_to_json(input_str)

print(json_obj)


{'translation': "S'inscrier"}
